<a href="https://colab.research.google.com/github/hieubkset/Colab-Notebooks/blob/master/overfit_and_underfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overfit and underfit**

Ở hai bài trước  [phân loại văn bản](https://ezcodin.com/tf03) và [regression](https://ezcodin.com/tf05), chúng ta đã thấy accuracy trên tập validation đạt đỉnh sau một vài epoch, sau đó nếu tiếp tục training nó sẽ bão hòa hoặc giảm.


Hiện tượng này được gọi là overfitting. Mặc dù overfitting có thể dẫn tới accuracy cao trên tập training nhưng mục tiêu của chúng ta là accuracy trên tập test (hay accuracy của model khi áp dụng vào thực tế).

Ngược lại với overfitting là underfitting. Nó là hiện tượng model có accuracy thấp trên cả tập train và tập test. Nguyên nhân có thể là: model quá đơn giản, bị regularize quá mức, hoặc là không đủ data cho training. 

Các kỹ thuật phổ biến để tránh overfitting bao gồm:

+ Thu thập thên data.
+ Data augmentation.
+ Giảm capacity .
+ Sử dụng weight regularization.
+ Sử dụng dropout.

Trong bài này, chúng ta sẽ tìm hiểu ảnh hưởng của capacity đến overfitting, cách sử dụng L2 regularization và dropout.

Mục tiêu:

+ Làm quen với kỹ thuật adaptive learning rate.
+ Học cách áp dụng một biến đối lên tất cả các phần tử trong dataset. Kỹ thuật này có thể ứng dụng trong nhiều trường hợp, ví dụ: data augumentation. Trong bài này, ta sẽ ứng dụng để tái cấu trúc dữ liệu của các phần tử.
+ Ôn tập kỹ thuật Early Stopping.
+ Tìm hiểu ảnh hưởng capacity.
+ Học cách sử dụng weight regularization và dropout.
+ Làm quen với cách sử dụng Tensorboard. Một công cụ rất hiệu quả để đánh giá overfitting và underfitting.

## Chương trình

#### **1. Khai báo các thư viện**

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [0]:
from  IPython import display
from matplotlib import pyplot as plt
import numpy as np
import pathlib
import shutil
import tempfile


Chúng ta sẽ sử dụng một số API từ [Tensorflow Docs](https://github.com/tensorflow/docs) như:

+ `tfdocs.modeling.EpochDots`: để tránh việc in quá nhiều log nếu training nhiều epoch.
+ `tfdocs.plots.HistoryPlotter`: để vẽ learning curve.

In [0]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

**Khai báo thư mục để chứa tensorboard log:**

+ `tempfile.mkdtemp`: tạo một thư mục tạm
+ `pathlib.Path`: cho phép tạo các đường dẫn theo cú pháp đơn giản
+ `shutil.rmtree`: xóa một thư mục (để đảm bảo các log cũ sẽ bị xóa) 

In [0]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)


### **2. Chuẩn bị data**

Chúng ta sẽ sử dụng [Higgs Dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS) bao gồm 11&#x202F;000&#x202F;000 example, mỗi example có 28 feature, và được gán nhãn 0 hoặc 1.

**Download data:**

In [0]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')

**Đọc data:**

In [0]:
FEATURES = 28

Sử dụng `tf.data.experimental.CsvDataset` để đọc dữ liệu trực tiếp từ tệp csv ở dạng nén.

In [0]:
ds = tf.data.experimental.CsvDataset(gz, [float(),]*(FEATURES+1), compression_type="GZIP")

**Tái cấu trúc dữ liệu của mỗi example:**

Mỗi phần tử của `ds` là một list gộp chung cả label và feature. Ta cần định dạng lại cấu trúc dữ liệu sao cho mỗi phần tử là một tuple của *features* và *label*. 

In [0]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

Tiếp theo sử dụng hàm `map` để áp dụng hàm `pack_row` lên tất cả các example. 

Đầu tiên, ta tạo batch với kích thước mỗi batch là 10000. Trong quá trình tạo batch, áp dụng hàm `pack_row` lên từng example trong batch. Sau đó, unbatch lại như lúc đầu.

In [0]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

Note: Cách áp dụng kỹ thuật *data agumentation* là hoàn toàn tương tự. Ta định nghĩa một hàm augmentation. Sau đó dùng hàm `map` để áp dụng lên tất cả các phần tử trong dataset. 

**Explore data:**

In [0]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

Sử dụng 1000 example đầu tiên cho validation và 10 000 example còn lại cho training.

In [0]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

Chúng ta sử dụng hàm `Dataset.skip` và `Dataset.take` để phân chia tập train và tập validation. Sử dụng `Dataset.cache` để đảm bảo không cần đọc lại dữ liệu từ file mỗi epoch:

In [0]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

**Tạo batch:**

In [0]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

### **3. Ảnh hưởng của capacity đến overfitting**

Cách đơn giản nhất để tránh overfitting là bắt đầu với một model đơn giản: một model với ít parameter (được xác định bởi số layer và số unit của mỗi layer). Thuật ngữ tiếng anh để chỉ số lượng parameter của model là capacity (*từ gốc là network's capacity hoặc model's capacity*).

Về mặt lý thuyết, một model với nhiều parameter sẽ có khả năng ghi nhớ tốt hơn. Nó sẽ dễ dàng học ánh xạ $x \rightarrow y$ (hay $features \rightarrow label$) mà thiếu đi sự tổng quát hóa. Điều này dẫn tới kết quả không tốt khi model gặp các dữ liệu không có trong tập training.

Tuy nhiên, không có một công thức để xác định capacity thích hợp. Chúng ta phải tiến hành nhiều thí nghiệm để lựa chọn: đầu tiên bắt đầu với model có capacity nhỏ sau đó tăng dần cho tới bắt gặp hiện tượng overfitting bằng cách quan sát validation loss.

Trong phần này, chúng ta sẽ so sánh 4 model với capacity tăng dần: **Tiny mode**, **Smal model**, **Medium model** và **Large model**. Chúng ta cũng sẽ sử dụng cùng một *training strategy* cho cả 4 model.

**Khai báo training stategy**

*(1) Sử dụng Optimizer và Learning Rate giống nhau*

Các model thường cho accuracy tốt hơn nếu learning rate được giảm từ từ trong quá trình training. Kỹ thuật này được gọi là Adaptive Learning Rate.
Chúng ta sẽ sử dụng InverseTimeDecay từ `optimizers.schedules`:

In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

`InverseTimeDecay` có công thức như sau:

$$decayed\_learning\_rate = learning\_rate / (1 + decay\_rate * t)$$

Cụ thể, chúng ta đã sử dụng: $learning\_rate=0.001$, $decay\_rate=1$. Như vậy, `learning rate` sẽ giảm đi $1/2$ sau 1000 epoch và $1/3$ sau 2000 epoch.

Đoạn code sau sẽ vẽ một đồ thị minh họa sự thay đổi của `learning rate`:

In [0]:
step = np.linspace(0, 1e5)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')


*(2) Sử dụng chung các callback*

+ Để tránh hiển thị quá nhiều log khi train model trong nhiều epoch, ta sử dụng `tfdocs.EpochDots`. Nó sẽ chỉ in ra log sau mỗi 100 epoch.

+ Sử dụng `callbacks.EarlyStopping` để tránh việc training lâu không cần thiết. Hàm callback này sẽ theo dõi `val_binary_crossentropy`. Nếu đại lượng này không giảm trong liên tiếp 200 epoch, quá trình training sẽ dừng lại. 

+ Sử dụng `callbacks.TensorBoard` để theo dõi log trong quá trình training. Một công cụ rất hiệu quả để đánh giá overfitting và underfitting.



In [0]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

*(3) Sử dụng cùng thiết lập cho `Model.compile` và `Model.fit`*

In [0]:
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[
                  tf.keras.losses.BinaryCrossentropy(
                      from_logits=True, name='binary_crossentropy'),
                  'accuracy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(name),
    verbose=0)
  return history

**Khai báo `size_histories` để lưu tất cả các history:**

In [0]:
size_histories = {}

#### **3.1. Tiny model**

Sử dụng 1 hidden layer với 16 unit:

In [0]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1)
])

In [0]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

**Vẽ learning curve:**

In [0]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
plt.ylim([0.5, 0.7])

#### **3.2. Small model**

Sử dụng 2 hidden layer với 16 unit:

In [0]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(16, activation='elu'),
    layers.Dense(1)
])

In [0]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

#### **3.3. Medium model**

Sử dụng 3 hidden layer với 64 unit:

In [0]:
medium_model = tf.keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1)
])

In [0]:
size_histories['Medium']  = compile_and_fit(medium_model, "sizes/Medium")

#### **3.4. Large model**

Sử dụng 4 hidden layer với 512 unit:

In [0]:
large_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(1)
])

In [0]:
size_histories['large'] = compile_and_fit(large_model, "sizes/Large")

### **4. Đánh giá overfitting từ learning curve**

In [0]:
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel("Epochs [Log Scale]")

Đường nét liền là training loss, nét đứt là validation loss. Các màu khác nhau đại diện cho các model với các capacity khác nhau.

Nhìn vào learning curve, ta thấy model có capacity càng lớn thì training loss càng nhỏ và càng nhanh overfitting (validation loss càng nhanh đạt cực tiểu và sau đó tăng).

### **5. Xem learning curve bằng TensorBoard**

Chúng ta đã quen với việc lưu lại các metric như loss, accuracy, v.v. trong training. Sau khi kết thức training, dùng `matplotlib` để vẽ learning curve. Tuy nhiên, các này có nhược điểm là ta không thể đánh giá model ngay trong quá trình training.

`Tensorboard` là một công cụ giải quyết vấn đề này. Trong quá trình training, Tensorboard Callback được gọi định kỳ (sau một số step nào đó), nó sẽ lưu các metric vào các file log trong một thư mục do chúng ta chỉ định. Ta có thể đọc các file này ngay trong quá trình training và vẽ các learning curve để theo dõi sự thay đổi của các metric.




**Để mở tensorboard trong notebook:**

Tham số `logdir` cho biết thư mục chứa các file log.

In [0]:
%load_ext tensorboard
%tensorboard --logdir {logdir}/sizes

**Chia sẻ tensorboard lên [TensorBoard.dev](https://tensorboard.dev/)**:

Chúng ta cũng có thể chia sẻ tensorboard với mọi người bằng cách upload các file log lên TensorBoard.dev.

In [0]:
!tensorboard dev upload --logdir  {logdir}/sizes

Lưu ý: Lệnh này yêu cầu xác thực tài khoản Google.

**Xem một tensoboard được chia sẻ lên [TensorBoard.dev](https://tensorboard.dev/)**:

In [0]:
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

## **6. Áp dụng phương pháp tránh overfitting**

Chúng ta sẽ sao lưu các log của Tiny model để so sánh với các kết quả khi áp dụng phương pháp tránh overfitting.

In [0]:
shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [0]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

### **6.1. Sử dụng Weight Regularization**



Một trong các cách tránh overfitting là sử dụng weight regularization. Có hai loại regularization phổ biến là:

* [L1 regularization](https://developers.google.com/machine-learning/glossary/#L1_regularization):

$$\theta^* = \arg min _ \theta \left( loss + \lambda \sum _ i |\theta _ i| \right)$$

* [L2 regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization):

$$\theta^* = \arg min _ \theta \left( loss + \lambda \sum _ i \theta _ i ^ 2 \right)$$

Trong thực tế L2-regularization được sử dụng nhiều hơn. 

In [0]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)
])

regularizer_histories['l2'] = compile_and_fit(l2_model, "regularizers/l2")

`l2(0.001)` tức $\lambda = 0.001$




In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Từ đồ thị trên ta thấy L2-regularization giúp giảm vấn đề overfitting trên Large model.

### **6.2. Sử dụng Dropout**

Dropout là một kỹ thuật tránh overfitting được sử dụng nhiều nhất được đề xuất bởi Hinton và các sinh viên tại đại học Toronto.

Ý tưởng của Dropout là ngẫu nhiên đóng, mở các node trong quá trình training. Điều này khiến network không thể chỉ dựa vào một vài node để cho ra kết quả, thay vào đó mỗi node phải học một feature có ý nghĩa.

Lưu ý: Ví dụ dưới đây, ta thêm vào network các dropout layer. Tuy nhiên, chúng chỉ được dùng trong quá trình training. Trong testing, các dropout layer sẽ không có tác dụng.

In [0]:
dropout_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Cả L2 regularization và Dropout giúp cải thiện chất lượng của Large model. Tuy nhiên kết quả vẫn còn hạn chế.



### **6.3. Kết hợp L2 và dropout**

In [0]:
combined_model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['combined'] = compile_and_fit(combined_model, "regularizers/combined")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Việc kết hợp các phương pháp tránh overfitting thông thường sẽ hiệu quả nhất.